In [1]:
import threading, itertools
import pyo
import time
import pyo as po
from pydub import AudioSegment
import sys,os,glob,time, signal
import numpy as np
import networkx as nx

In [2]:
# Set number of input and output channels
# Ch. 0,1,2,3 -> to speakers
# Ch. 4,5,6,7 -> to radio transmitters
nch = 12
ich = 0
# Set audio server
s = po.Server(nchnls=nch,ichnls=ich,audio='jack').boot()
s.start()

Pyo warning: Buffer size set to Jack engine buffer size: 1024.


### Read the samples and create the dictionaries

In [3]:
def importSoundfiles(dirpath='./',filepath='./',mult=0.1,gain=1.0):

    # reading wavefiles

    # files
    try:
        obj = [None]*len(glob.glob(dirpath+filepath))
        fil = [None]*len(glob.glob(dirpath+filepath))
        n=0
        for file in glob.glob(dirpath+filepath):
            fil[n] = file
            n += 1
        for i in range(len(glob.glob(dirpath+filepath))):
            obj[i] = po.SfPlayer(fil[i],mul=mult*gain)
    except:
        print('error in file reading',dirpath+filepath)
        pass

    return(obj,fil,mult)

In [4]:
def chinese_postman(graph,starting_node=None,verbose=False):
        
    def get_shortest_distance(graph, pairs, edge_weight_name):
        return {pair : nx.dijkstra_path_length(graph, pair[0], pair[1], edge_weight_name) for pair in pairs}

    def create_graph(node_pairs_with_weights, flip_weight = True):
        graph = nx.Graph()
        for k,v in node_pairs_with_weights.items():
            wt = -v if flip_weight else v
            graph.add_edge(k[0], k[1], **{'distance': v, 'weight': wt})
        return graph

    def create_new_graph(graph, edges, starting_node=None):
        g = nx.MultiGraph()
        for edge in edges:
            aug_path  = nx.shortest_path(graph, edge[0], edge[1], weight="distance")
            aug_path_pairs  = list(zip(aug_path[:-1],aug_path[1:]))

            for aug_edge in aug_path_pairs:
                aug_edge_attr = graph[aug_edge[0]][aug_edge[1]]
                g.add_edge(aug_edge[0], aug_edge[1], attr_dict=aug_edge_attr)
        for edge in graph.edges(data=True):
            g.add_edge(edge[0],edge[1],attr_dict=edge[2:])
        return g

    def create_eulerian_circuit(graph, starting_node=starting_node):
        return list(nx.eulerian_circuit(graph,source=starting_node))
    
    odd_degree_nodes = [node for node, degree in dict(nx.degree(graph)).items() if degree%2 == 1]
    odd_degree_pairs = itertools.combinations(odd_degree_nodes, 2)
    odd_nodes_pairs_shortest_path = get_shortest_distance(graph, odd_degree_pairs, "distance")
    graph_complete_odd = create_graph(odd_nodes_pairs_shortest_path, flip_weight=True)
    if verbose:
        print('Number of nodes (odd): {}'.format(len(graph_complete_odd.nodes())))
        print('Number of edges (odd): {}'.format(len(graph_complete_odd.edges())))
    odd_matching_edges = nx.algorithms.max_weight_matching(graph_complete_odd, True)
    if verbose: print('Number of edges in matching: {}'.format(len(odd_matching_edges)))
    multi_graph = create_new_graph(graph, odd_matching_edges)

    return(create_eulerian_circuit(multi_graph, starting_node))


## Import sounds

In [5]:
# I Coro
S_obj,S_fil,_ = importSoundfiles(dirpath='../SOUNDS/1mo-CORO/',
                                     filepath='/1-SOPRANO/*.wav',mult=1.0,gain=1.0)
A_obj,A_fil,_ = importSoundfiles(dirpath='../SOUNDS/1mo-CORO/',
                                     filepath='/2-CONTRALTO/*.wav',mult=1.0,gain=1.0)
T_obj,T_fil,_ = importSoundfiles(dirpath='../SOUNDS/1mo-CORO/',
                                     filepath='/3-TENORE/*.wav',mult=1.0,gain=1.0)
B_obj,B_fil,_ = importSoundfiles(dirpath='../SOUNDS/1mo-CORO/',
                                     filepath='/4-BASSO/*.wav',mult=1.0,gain=1.0)

# II coro
II_obj,II_fil,_ = importSoundfiles(dirpath='../SOUNDS/2do-CORO/',
                                     filepath='/*.wav',mult=1.0,gain=1.0)

# III coro
III_obj,III_fil,_ = importSoundfiles(dirpath='../SOUNDS/3zo-CORO/',
                                     filepath='/*.wav',mult=1.0,gain=2.0)

# IV coro
IV_obj,IV_fil,_ = importSoundfiles(dirpath='../SOUNDS/4to-CORO/',
                                     filepath='/*.wav',mult=1.0,gain=1.0)

# combine "guerrilla" sounds in list
g_obj = [II_obj,III_obj,IV_obj]
g_fil = [II_fil,III_fil,IV_fil]

In [6]:
# check that all files are in wav format at 44100, else convert
for file in glob.glob('../SOUNDS/VOCI/*'):
    if file.split('.')[-1] != 'wav':
        m4a_file = file
        wav_filename = m4a_file[:-len(file.split('.')[-1])]+'wav'
        track = AudioSegment.from_file(m4a_file,  format= 'm4a')
        track = track.set_frame_rate(44100)
        file_handle = track.export(wav_filename, format='wav')

In [7]:
# voices
v_obj,v_fil,_ = importSoundfiles(dirpath='../SOUNDS/VOCI/',
                                     filepath='/*.wav',mult=1.0,gain=1.0)

### Define network and path

In [8]:
# This function plays the instrumental background - I coro
def playPart(obj,fil,mul,nch):
    global stop_threads
    while True:
        if stop_threads:
            break
        Gx = nx.barabasi_albert_graph(len(fil),2)
        chino = chinese_postman(Gx,None,verbose=False)
        seq = [chino[0][0]]
        for s in range(1,len(chino)):
            seq.append(chino[s][1])
            if stop_threads:
                break
        for n in range(len(seq)):
            obj[seq[n]].out(nch,0).setMul(mul)
            time.sleep(po.sndinfo(fil[seq[n]])[1])
            if stop_threads:
                break
    return

# This function plays the "guerrilla" episodes in the II, III and IV coro on radios
def playRadioGuerrilla(obj,fil,mul,nch=np.arange(4,8),delay=30):
    global stop_threads
    while True:
        if stop_threads:
            break
        for i in range(3):
            if stop_threads:
                    break
            time.sleep(delay*(np.random.rand()+1))
            Gx = nx.barabasi_albert_graph(len(fil[i]),2)
            chino = chinese_postman(Gx,None,verbose=False)
            seq = [chino[0][0]]
            for s in range(1,len(chino)):
                seq.append(chino[s][1])
                if stop_threads:
                    break
            for n in range(len(seq)):
                ch = int(np.random.choice(nch))
                obj[i][seq[n]].out(ch,0).setMul(mul)
                time.sleep(po.sndinfo(fil[i][seq[n]])[1])
                if stop_threads:
                    break
    return

# This function plays the voices to radios
def playVoices(obj,fil,mul,nch=np.arange(8,12)):
    global stop_threads
    while True:
        if stop_threads:
            break
        Gx = nx.barabasi_albert_graph(len(fil),2)
        chino = chinese_postman(Gx,None,verbose=False)
        seq = [chino[0][0]]
        for s in range(1,len(chino)):
            seq.append(chino[s][1])
            if stop_threads:
                break
        for n in range(len(seq)):
            ch = int(np.random.choice(nch))
            obj[seq[n]].out(ch,0).setMul(mul)
            time.sleep(po.sndinfo(fil[seq[0]])[1]/(1+np.random.rand()))
            if stop_threads:
                break
    return

## Orchestra

In [9]:
stop_threads=False
# I coro
threading.Thread(target=playPart,args=(S_obj,S_fil,0.1,0)).start()
threading.Thread(target=playPart,args=(A_obj,A_fil,0.1,1)).start()
threading.Thread(target=playPart,args=(T_obj,T_fil,0.1,2)).start()
threading.Thread(target=playPart,args=(B_obj,B_fil,0.1,3)).start()
threading.Thread(target=playPart,args=(S_obj,S_fil,0.1,3)).start()
threading.Thread(target=playPart,args=(A_obj,A_fil,0.1,2)).start()
threading.Thread(target=playPart,args=(T_obj,T_fil,0.1,1)).start()
threading.Thread(target=playPart,args=(B_obj,B_fil,0.1,0)).start()
# II,III & IV coro (radio guerrilla)
threading.Thread(target=playRadioGuerrilla,args=(g_obj,g_fil,0.2)).start()
# voices
threading.Thread(target=playVoices,args=(v_obj,v_fil,0.4)).start()

In [10]:
stop_threads = True
print('threads killed')

threads killed


In [14]:
threading.active_count()

8

## Timed threading to get new audio files

In [9]:
from datetime import datetime

In [10]:
def check_time():
    global stop_time
    before = datetime.now()
    while True:
        if stop_time:
            break
        # datetime object containing current date and time
        now = datetime.now()
        if now.strftime("%M") != before.strftime("%M"):
            before = now.now()
            print(before.strftime("%M"))

In [12]:
stop_time = False
threading.Thread(target=check_time).start()

2424



In [13]:
stop_time = True

In [ ]:
threading.active_count()